In [ ]:
import  pandas as pd
import numpy as np
import seaborn as sbn
import matplotlib.pyplot as plt
import matplotlib.ticker as tck

In [ ]:
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
df_planning_2018 = pd.read_csv("../data/releases_2018/planning_2018.csv")
df_planning_2019 = pd.read_csv("../data/releases_2019/planning_2019.csv")
df_planning_2020 = pd.read_csv("../data/releases_2020/planning_2020.csv")
df_planning_2021 = pd.read_csv("../data/releases_2021/planning_2021.csv")
df_planning = pd.concat([df_planning_2018, df_planning_2019, df_planning_2020, df_planning_2021], axis=0)

In [ ]:
df_tender_2018 = pd.read_csv("../data/releases_2018/tender_2018.csv")
df_tender_2019 = pd.read_csv("../data/releases_2019/tender_2019.csv")
df_tender_2020 = pd.read_csv("../data/releases_2020/tender_2020.csv")
df_tender_2021 = pd.read_csv("../data/releases_2021/tender_2021.csv")
df_tender = pd.concat([df_tender_2018, df_tender_2019, df_tender_2020, df_tender_2021], axis=0)

In [ ]:
df_award_2018 = pd.read_csv("../data/releases_2018/awards_2018.csv")
df_award_2019 = pd.read_csv("../data/releases_2019/awards_2019.csv")
df_award_2020 = pd.read_csv("../data/releases_2020/awards_2020.csv")
df_award_2021 = pd.read_csv("../data/releases_2021/awards_2021.csv")
df_award = pd.concat([df_award_2018, df_award_2019, df_award_2020, df_award_2021], axis=0)

In [ ]:
df_contract_2018 = pd.read_csv("../data/releases_2018/contracts_2018.csv")
df_contract_2019 = pd.read_csv("../data/releases_2019/contracts_2019.csv")
df_contract_2020 = pd.read_csv("../data/releases_2020/contracts_2020.csv")
df_contract_2021 = pd.read_csv("../data/releases_2021/contracts_2021.csv")
df_contract = pd.concat([df_contract_2018, df_contract_2019, df_contract_2020, df_contract_2021], axis=0)

In [ ]:
df_release_2018 = pd.read_csv("../data/releases_2018/releases_2018.csv")
df_release_2019 = pd.read_csv("../data/releases_2019/releases_2019.csv")
df_release_2020 = pd.read_csv("../data/releases_2020/releases_2020.csv")
df_release_2021 = pd.read_csv("../data/releases_2021/releases_2021.csv")
df_release = pd.concat([df_release_2018, df_release_2019, df_release_2020, df_release_2021], axis=0)

In [ ]:
df_supplier_2018 = pd.read_csv("../data/releases_2018/suppliers_2018.csv")
df_supplier_2019 = pd.read_csv("../data/releases_2019/suppliers_2019.csv")
df_supplier_2020 = pd.read_csv("../data/releases_2020/suppliers_2020.csv")
df_supplier_2021 = pd.read_csv("../data/releases_2021/suppliers_2021.csv")
df_supplier = pd.concat([df_supplier_2018, df_supplier_2019, df_supplier_2020, df_supplier_2021], axis=0)

In [ ]:
res_tender = df_tender.loc[:, ["ocid", "procuringEntity_name", "procurementMethod", "value_amount"]]
res_award = df_award.loc[:, ["ocid", "status"]]
res1 = res_tender.merge(res_award, left_on="ocid", right_on="ocid", how="left")
res1

In [ ]:
res1 = res1[(res1["status"] == "active")]
res1

In [ ]:
def competitive(x):
    if (x == "direct"):
        return "No competitive"
    return "Competitive"

In [ ]:
res1["procurementMethod"] = res1["procurementMethod"].apply(competitive)
res1

In [ ]:
res1 = res1.groupby(["procuringEntity_name", "procurementMethod"])["ocid"].count().reset_index()
res1["total_procesos"] = res1.groupby("procuringEntity_name")["ocid"].transform("sum")
res1.columns = ["Licitante", "Metodo", "Cantidad", "Total"]
res1 = res1[~(res1["Cantidad"] == res1["Total"])]
res1["Porcentaje"] = res1["Cantidad"] / res1["Total"]
res1 = res1[(res1["Metodo"] == "No competitive") & (res1["Porcentaje"] > 0.75) & (res1["Cantidad"] > 80)]
res1.reset_index(drop=True)

In [ ]:
def filtrar(x: str):
    x = x.upper()
    if x.startswith("CENTRO CLINICO"):
        return "CENTRO CLINICO"
    if x.startswith("HOSPITAL"):
        return "HOSPITAL"
    if x.startswith("GOBIERNO AUTONOMO"):
        return "GOBIERNO AUTONOMO"
    if x.startswith("DIRECCION DISTRITAL"):
        return "DIRECCION DISTRITAL"
    if x.startswith("CENTRO DE SALUD"):
        return "CENTRO DE SALUD"
    if x.startswith("MINISTERIO"):
        return "MINISTERIO"
    return x

In [ ]:
res1["Licitante"] = res1["Licitante"].apply(filtrar)
res1 = res1.groupby("Licitante").agg({"Porcentaje": ["mean", "count"]})
res1.columns = res1.columns.droplevel()
res1.columns = ["Porcentaje", "Frecuencia"]
res1.sort_values("Frecuencia", ascending=False)
res1.head()

In [ ]:
dib1 = res1.sort_values("Porcentaje", ascending=False)[:10]
dib2 = res1.sort_values("Frecuencia", ascending=False)[:10]
sbn.set_style("darkgrid")
fig,axs = plt.subplots(2,1,figsize=(16,16))
sbn.barplot(data=dib1, x="Porcentaje", y=dib1.index, ax=axs[0])
sbn.barplot(data=dib2, x="Frecuencia", y=dib2.index, ax=axs[1])
_ = plt.setp(axs[0].set_title("Top 10 porcentaje de procesos no competitivos"))
_ = plt.setp(axs[1].set_title("Frecuencia de entidad de la misma categoría"))
fig.tight_layout()
plt.savefig("../output/redflag-jhony1.jpg")

In [ ]:
res2_tender = df_tender[["ocid", "procuringEntity_name"]]
res2_award = df_award[["ocid", "id", "amount",]]
res2_contract = df_contract[["ocid", "id", "amount"]]

In [ ]:
res2 = res2_award.merge(res2_contract, left_on="ocid", right_on="ocid", how="left", suffixes=("_aw", "_ctr"))
res2 = res2[~(res2["amount_ctr"].isnull()) & ~(res2["amount_aw"].isnull())]
res2 = res2.merge(res2_tender, left_on="ocid", right_on="ocid", how="left")

In [ ]:
res2["diferencia_absoluta"] = np.abs(res2["amount_aw"] - res2["amount_ctr"])
res2["diferencia_relativa"] = res2["diferencia_absoluta"] / res2["amount_aw"]
res2.head()

In [ ]:
res2 = res2[(res2["diferencia_relativa"] > 0) & (res2["diferencia_relativa"] <= 0.8)]
res2 = res2.groupby("procuringEntity_name").agg({"diferencia_relativa" : "mean", "diferencia_absoluta" : "mean"}).reset_index()
res2.head()

In [ ]:
res2.columns = ["Licitante", "Diferencia relativa", "Diferencia absoluta"]
res2["Licitante"] = res2["Licitante"].str.upper().str[:60] + "..."
res2.head()

In [ ]:
dib2_rel = res2.sort_values("Diferencia relativa", ascending=False)
dib2_abs = res2.sort_values("Diferencia absoluta", ascending=False)
sbn.set_style("darkgrid")
fig,axs = plt.subplots(2,1,figsize=(16,10))
sbn.barplot(data=dib2_rel[:10], x="Diferencia relativa", y="Licitante", ax=axs[0])
sbn.barplot(data=dib2_abs[:10], x="Diferencia absoluta", y="Licitante", ax=axs[1])
plt.setp(axs[0].set_title("Top 10 diferencia relativa entre precio adjudicado y precio declarado en la implementación"))
plt.setp(axs[1].set_title("Top 10 diferencia absoluta entre precio adjudicado y precio declarado en la implementación"))
axs[1].get_xaxis().get_major_formatter().set_scientific(False)
fig.tight_layout()
plt.savefig("../output/redflag-jhony2.jpg")

In [ ]:
res3_tender = df_tender[["ocid", "procuringEntity_name", "value_amount", "procurementMethod"]]
res3_award = df_award[["ocid", "status", "id"]]
res3_supplier = df_supplier[["award_id", "name"]]
tmp = res3_award.merge(res3_supplier, left_on="id", right_on="award_id", how="left")
res3 = res3_tender.merge(tmp, left_on="ocid", right_on="ocid", how="left")
res3.head()

In [ ]:
def filtrar(row):
    porc = row["Porcentaje"]
    if porc < 0.1:
        row["Licitador"] = "OTROS"
    row["Licitante"] = row["Licitante"].upper()[:70]
    row["Licitador"] = row["Licitador"].upper()
    return row

In [ ]:
result3 = res3.groupby(["procuringEntity_name", "name"])["id"].count().reset_index()
result3["total_procesos"] = result3.groupby(["procuringEntity_name"])["id"].transform("sum")
result3["unicos"] = result3.groupby(["procuringEntity_name"])["id"].transform("count")
result3["porcentaje"] = result3["id"] / result3["total_procesos"]
result3 = result3[(result3["total_procesos"] != result3["id"]) & (result3["unicos"] >= 40)]
filtro3 = result3.groupby(["procuringEntity_name"])["porcentaje"].transform("max") >= 0.6
result3 = result3[filtro3].reset_index(drop=True)
result3.columns = ["Licitante", "Licitador", "Cantidad", "Total", "Uniques", "Porcentaje"]
result3 = result3.apply(filtrar, axis=1)
result3.head()

In [ ]:
result3 = result3.groupby(["Licitante", "Licitador"])["Porcentaje"].sum().reset_index()
result3.head()

In [ ]:
result3.sort_values("Porcentaje", ascending=False).head()

In [ ]:
dib3 = pd.pivot_table(result3, index="Licitante", columns="Licitador", fill_value=0)
dib3.columns = dib3.columns.droplevel()

In [ ]:
sbn.set_style("darkgrid")
fig, axs = plt.subplots(figsize=(16, 16))
_ = sbn.heatmap(dib3, yticklabels=dib3.index, xticklabels=dib3.columns,
                ax=axs, annot=True, fmt=".2f", cmap="YlOrBr")
axs.set_title(
    "Porcentaje de procesos acreditados Licitante vs Licitador. Licitantes con 40 clientes únicos.")
_ = plt.xticks(rotation=90)
fig.tight_layout()
plt.savefig("../output/redflag-jhony3.jpg")